# Payment 관련 feature engineering

In [1]:
import warnings
warnings.simplefilter('ignore')

----

# Train

In [3]:
# train data
label = pd.read_csv("~/documents/chaser_data/train_label.csv")
print("shape of label: ", label.shape)

shape of label:  (100000, 2)


In [5]:
payment = pd.read_csv("~/documents/chaser_data/train_payment.csv")
payment.tail()

,payment_week,acc_id,payment_amount
799995,4,7b08dab0401bf3f2ab44bfdfa0b0c2b93d6bed5d8e4350...,-0.149898
799996,5,7b08dab0401bf3f2ab44bfdfa0b0c2b93d6bed5d8e4350...,-0.149898
799997,6,7b08dab0401bf3f2ab44bfdfa0b0c2b93d6bed5d8e4350...,-0.149898
799998,7,7b08dab0401bf3f2ab44bfdfa0b0c2b93d6bed5d8e4350...,-0.149898
799999,8,7b08dab0401bf3f2ab44bfdfa0b0c2b93d6bed5d8e4350...,-0.149898


## 1. week별 payment amount 변수 생성
- 유저별 각 주의 payment amount를 pivot하여 주별 변수로 분리

In [6]:
payment_amount = payment.pivot("acc_id", "payment_week", "payment_amount")
payment_amount.rename(columns={1:"pay_amount_w1", 2:"pay_amount_w2", 3:"pay_amount_w3", 
                               4:"pay_amount_w4", 5:"pay_amount_w5", 6:"pay_amount_w6",
                               7:"pay_amount_w7", 8:"pay_amount_w8"}, inplace=True)
payment_amount.reset_index(inplace=True)
print("shape of payment_amount:", payment_amount.shape)
payment_amount.tail()

shape of payment_amount: (100000, 9)


payment_week,acc_id,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8
99995,fffca0aabb4cab09c210bbab2fd6f22ac9d6b78278e6fa...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898
99996,fffe5968f76728edd28b7f458e071f8c598a93112506fd...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898
99997,fffe913627c68c59eaea10e318c599aba4018dcecb1657...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898
99998,ffffa8aae3cf71b6ac34b24f3dd1346d96931942164133...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898
99999,ffffc888fd718bee1107d8e39d3c3553604fea4d6eb691...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898


## 2. payment가 있었던 week 수 cnt 변수 생성
- payment가 있었던 주만 보기 위해 수정 전 데이터 사용 
    - 수정 전에는 payment가 있었던 주의 자료만 존재, 수정 후에는 1-8주 row가 모두 존재

In [7]:
payment_week = pd.read_csv("~/documents/chaser_data/train_payment_old.csv")
payment_week.tail()

,payment_week,acc_id,payment_amount
40701,4,678266ffac502a387d45bdcfbd4865cbfde91a3e651459...,1.006096
40702,7,06006fd6827b554b7d72d29c9c4d49d959b4a3fc66d900...,0.277290
40703,3,715f2b110c1480e407ee2980e5c8eb13770ab43a29516a...,-0.659747
40704,7,715f2b110c1480e407ee2980e5c8eb13770ab43a29516a...,-0.336990
40705,8,715f2b110c1480e407ee2980e5c8eb13770ab43a29516a...,-0.243287


In [8]:
cnt_payment_week = payment_week.loc[:,["acc_id","payment_week"]].groupby("acc_id").size().reset_index(name="cnt_payment_week")
cnt_payment_week.tail()

,acc_id,cnt_payment_week
12297,ffe7ffe81fb8a9369ecb8aed4aa736f64dc145d131fabf...,1
12298,ffeb3ba7c7f2195efe469d04e062606919d31818536e9f...,2
12299,ffec0742aea3302a4157769d95c13b47f22d538e3396e4...,1
12300,fff39e6b6c5e8fb975824a01f86197bb53ca94b295001b...,4
12301,fffbdf6a660029ddbe8c78af1f56964e98bfc1f8e3e49e...,4


Comments
- 한 번이라도 payment를 한 유저는 train 10만 명 중 12302명
- 한 번이라도 payment를 한 유저는 test 4만 명 중 4950명

## 3. 마지막 payment가 있었던 week 변수 생성
- 유저가 마지막으로 payment를 했던 주가 1-8주 중 몇 번째 주인지 나타내는 변수 생성

In [9]:
result2 = payment_week.loc[:,["acc_id","payment_week"]].groupby("acc_id").agg(["max"]).reset_index()

last_payment_week = pd.DataFrame(columns=["acc_id", "last_payment_week"])
last_payment_week["acc_id"] = result2["acc_id"]
last_payment_week["last_payment_week"] = result2["payment_week"]["max"]
last_payment_week.describe()

,last_payment_week
count,12302.000000
mean,6.662169
std,1.776560
min,1.000000
25%,6.000000
50%,7.000000
75%,8.000000
max,8.000000


## 4. week payment amount의 max, total 변수 생성
- 유저의 주별 payment amount 중 최대값, 1-8주간의 payment amount의 합

In [10]:
result = payment.loc[:,["acc_id","payment_amount"]].groupby("acc_id").agg(["max","sum"]).reset_index()
payment = pd.DataFrame(columns=["acc_id", "max_payment_amount", "total_payment_amount"])
payment["acc_id"] = result["acc_id"]
payment["max_payment_amount"] = result["payment_amount"]["max"]
payment["total_payment_amount"] = result["payment_amount"]["sum"]
payment.tail()

,acc_id,max_payment_amount,total_payment_amount
99995,fffca0aabb4cab09c210bbab2fd6f22ac9d6b78278e6fa...,-0.149898,-1.199188
99996,fffe5968f76728edd28b7f458e071f8c598a93112506fd...,-0.149898,-1.199188
99997,fffe913627c68c59eaea10e318c599aba4018dcecb1657...,-0.149898,-1.199188
99998,ffffa8aae3cf71b6ac34b24f3dd1346d96931942164133...,-0.149898,-1.199188
99999,ffffc888fd718bee1107d8e39d3c3553604fea4d6eb691...,-0.149898,-1.199188


## 5. week payment의 표준편차 변수 생성
- 유저의 주별 payment amount의 표준편차를 구해, 유저가 주별로 고르게 결제했는지 몰아서 결제했는지를 나타내는 변수 생성

In [11]:
payment_amount["pay_amount_std"] = payment_amount.loc[:,"pay_amount_w1":"pay_amount_w8"].std(axis=1)
payment_amount.fillna(0, inplace=True)
payment_amount.tail()

payment_week,acc_id,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8,pay_amount_std
99995,fffca0aabb4cab09c210bbab2fd6f22ac9d6b78278e6fa...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.0
99996,fffe5968f76728edd28b7f458e071f8c598a93112506fd...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.0
99997,fffe913627c68c59eaea10e318c599aba4018dcecb1657...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.0
99998,ffffa8aae3cf71b6ac34b24f3dd1346d96931942164133...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.0
99999,ffffc888fd718bee1107d8e39d3c3553604fea4d6eb691...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.0


## 6. 하나의 dataframe으로 merge

In [12]:
label.tail()

,acc_id,label
99995,da6d33b03968d8e35821f6eb88ad22e12e37aa8867084e...,retained
99996,676c944f4b6ae63818b3cad824a61233690f16a2275d5d...,retained
99997,695e1f28e234fc4cc53085e332fa7a76d7895ca4cc745b...,retained
99998,0c87fabaad5542e533f958a1d6fd739993b94e95e00989...,retained
99999,47ff575cb94019df5695c5d81ec285b0d801607b2a8697...,retained


In [13]:
result = pd.merge(label, cnt_payment_week, how='outer').fillna(value=0)
result = pd.merge(result, last_payment_week, how='outer').fillna(value=0)
result = pd.merge(result, payment, how='outer')
result = pd.merge(result, payment_amount, how='outer')
result.tail()

,acc_id,label,cnt_payment_week,last_payment_week,max_payment_amount,total_payment_amount,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8,pay_amount_std
99995,da6d33b03968d8e35821f6eb88ad22e12e37aa8867084e...,retained,1.0,5.0,0.553227,-0.496063,-0.149898,-0.149898,-0.149898,-0.149898,0.553227,-0.149898,-0.149898,-0.149898,0.248592
99996,676c944f4b6ae63818b3cad824a61233690f16a2275d5d...,retained,0.0,0.0,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
99997,695e1f28e234fc4cc53085e332fa7a76d7895ca4cc745b...,retained,5.0,8.0,1.959477,2.527376,-0.114742,0.553227,0.553227,0.025883,-0.149898,-0.149898,-0.149898,1.959477,0.730407
99998,0c87fabaad5542e533f958a1d6fd739993b94e95e00989...,retained,0.0,0.0,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
99999,47ff575cb94019df5695c5d81ec285b0d801607b2a8697...,retained,4.0,8.0,6.881354,12.863316,-0.149898,1.607915,-0.149898,-0.149898,1.607915,3.365728,-0.149898,6.881354,2.485923


In [14]:
print(result.shape)

(100000, 15)


In [15]:
result.describe(include="all")

,acc_id,label,cnt_payment_week,last_payment_week,max_payment_amount,total_payment_amount,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8,pay_amount_std
count,100000,100000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
unique,100000,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,7b6cd83c2c7bacb1b175a24eedfe06b6b58ec3ac15b5f0...,month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,25000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,0.407060,0.81958,0.436842,-0.000979,-0.062886,-0.078420,0.055827,0.048050,-0.040545,0.029695,0.004998,0.042301,0.216597
std,NaN,NaN,1.275499,2.27525,2.264902,4.763623,0.617678,0.495593,1.213359,1.290024,0.776365,1.191789,0.973091,1.134096,0.834495
min,NaN,NaN,0.000000,0.00000,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
25%,NaN,NaN,0.000000,0.00000,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
50%,NaN,NaN,0.000000,0.00000,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
75%,NaN,NaN,0.000000,0.00000,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000


In [46]:
result.to_csv("../data/train_payment.csv", index=False)

----

# Test

In [16]:
# activity = pd.read_csv('../data/test_activity.csv')
activity = pd.read_csv('~/documents/chaser_data/test_activity.csv')
label = pd.DataFrame(list(activity['acc_id'].unique()))
label.columns = ['acc_id']
print("shape of label: ", label.shape)

shape of label:  (40000, 1)


In [4]:
# label.to_csv("~/documents/chaser_data/test_label.csv")

In [16]:
payment = pd.read_csv("~/documents/chaser_data/test_payment.csv")
print("shape of payment: ", payment.shape)
payment.tail()

shape of payment:  (320000, 3)


,payment_week,acc_id,payment_amount
319995,4,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,-0.149898
319996,5,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,14.510262
319997,6,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,2.627446
319998,7,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,-0.149898
319999,8,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,5.369634


## 1. week별 payment amount 변수 생성
- 유저별 각 주의 payment amount를 pivot하여 주별 변수로 분리

In [17]:
payment_amount = payment.pivot("acc_id", "payment_week", "payment_amount")
payment_amount.rename(columns={1:"pay_amount_w1", 2:"pay_amount_w2", 3:"pay_amount_w3", 
                               4:"pay_amount_w4", 5:"pay_amount_w5", 6:"pay_amount_w6",
                               7:"pay_amount_w7", 8:"pay_amount_w8"}, inplace=True)
payment_amount.reset_index(inplace=True)
print("shape of payment_amount:", payment_amount.shape)
payment_amount.tail()

shape of payment_amount: (40000, 9)


payment_week,acc_id,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8
39995,fff8bdad36759f50f58f1bb98f88d4023021779ca74ece...,0.553227,-0.079586,-0.149898,-0.149898,-0.149898,2.662602,1.607915,-0.149898
39996,fff8dfb008a887d5521f29b418e68982e233c27f8fe146...,-0.149898,-0.149898,-0.149898,-0.149898,0.553227,-0.149898,-0.149898,-0.149898
39997,fff93932cd1c91c38fda78a34b8f238ea395adbbb96e3f...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898
39998,fffba6445674e1b956dcd28644b55542bd29e44f72847f...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898
39999,fffd9c46a6762a7d4fdc43026abf75ab1a7b9352818f04...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898


## 2. payment가 있었던 week 수 cnt 변수 생성
- payment가 있었던 주만 보기 위해 수정 전 데이터 사용 
    - 수정 전에는 payment가 있었던 주의 자료만 존재, 수정 후에는 1-8주 row가 모두 존재

In [19]:
payment_week = pd.read_csv("~/documents/chaser_data/test_payment_old.csv")
payment_week.tail()

,payment_week,acc_id,payment_amount
16571,1,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,-0.347402
16572,2,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,-0.305756
16573,5,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,3.473628
16574,6,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,-0.045468
16575,8,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,0.766631


In [20]:
cnt_payment_week = payment_week.loc[:,["acc_id","payment_week"]].groupby("acc_id").size().reset_index(name="cnt_payment_week")
cnt_payment_week.tail()

,acc_id,cnt_payment_week
4945,ffd8077085ff2714e6b3d0a5e20501720d498d841414a5...,2
4946,ffdfa5460f7a0687631d91165fdf264a9a45a7612646c4...,4
4947,fff1416cec82334ad7320c466c8d777782cde14beb657a...,2
4948,fff8bdad36759f50f58f1bb98f88d4023021779ca74ece...,4
4949,fff8dfb008a887d5521f29b418e68982e233c27f8fe146...,1


Comments
- 한 번이라도 payment를 한 유저는 train 10만 명 중 12302명
- 한 번이라도 payment를 한 유저는 test 4만 명 중 4950명

## 3. 마지막 payment가 있었던 week 변수 생성
- 유저가 마지막으로 payment를 했던 주가 1-8주 중 몇 번째 주인지 나타내는 변수 생성

In [21]:
result2 = payment_week.loc[:,["acc_id","payment_week"]].groupby("acc_id").agg(["max"]).reset_index()

last_payment_week = pd.DataFrame(columns=["acc_id", "last_payment_week"])
last_payment_week["acc_id"] = result2["acc_id"]
last_payment_week["last_payment_week"] = result2["payment_week"]["max"]
last_payment_week.describe()

,last_payment_week
count,4950.000000
mean,6.687273
std,1.707430
min,1.000000
25%,6.000000
50%,7.000000
75%,8.000000
max,8.000000


## 4. week payment amount의 max, total 변수 생성
- 유저의 주별 payment amount 중 최대값, 1-8주간의 payment amount의 합

In [22]:
result = payment.loc[:,["acc_id","payment_amount"]].groupby("acc_id").agg(["max","sum"]).reset_index()
payment = pd.DataFrame(columns=["acc_id", "max_payment_amount", "total_payment_amount"])
payment["acc_id"] = result["acc_id"]
payment["max_payment_amount"] = result["payment_amount"]["max"]
payment["total_payment_amount"] = result["payment_amount"]["sum"]
payment.tail()

,acc_id,max_payment_amount,total_payment_amount
39995,fff8bdad36759f50f58f1bb98f88d4023021779ca74ece...,2.662602,4.144564
39996,fff8dfb008a887d5521f29b418e68982e233c27f8fe146...,0.553227,-0.496063
39997,fff93932cd1c91c38fda78a34b8f238ea395adbbb96e3f...,-0.149898,-1.199188
39998,fffba6445674e1b956dcd28644b55542bd29e44f72847f...,-0.149898,-1.199188
39999,fffd9c46a6762a7d4fdc43026abf75ab1a7b9352818f04...,-0.149898,-1.199188


## 5. week payment의 표준편차 변수 생성
- 유저의 주별 payment amount의 표준편차를 구해, 유저가 주별로 고르게 결제했는지 몰아서 결제했는지를 나타내는 변수 생성

In [23]:
payment_amount["pay_amount_std"] = payment_amount.loc[:,"pay_amount_w1":"pay_amount_w8"].std(axis=1)
payment_amount.fillna(0, inplace=True)
payment_amount.tail()

payment_week,acc_id,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8,pay_amount_std
39995,fff8bdad36759f50f58f1bb98f88d4023021779ca74ece...,0.553227,-0.079586,-0.149898,-0.149898,-0.149898,2.662602,1.607915,-0.149898,1.064353
39996,fff8dfb008a887d5521f29b418e68982e233c27f8fe146...,-0.149898,-0.149898,-0.149898,-0.149898,0.553227,-0.149898,-0.149898,-0.149898,0.248592
39997,fff93932cd1c91c38fda78a34b8f238ea395adbbb96e3f...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
39998,fffba6445674e1b956dcd28644b55542bd29e44f72847f...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
39999,fffd9c46a6762a7d4fdc43026abf75ab1a7b9352818f04...,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000


## 6. 하나의 dataframe으로 merge

In [24]:
label.tail()

,acc_id
39995,d048b24f5205a50916339d24fc9f7b1c69352d2ee10f1c...
39996,ad5b3ef19e64aceb9ffea55310607ba62146218e616a83...
39997,972114a3bb57c377d162fb66b9198a2eec40de9576af51...
39998,3d10d680df3d5fa2e370997c18274bad7355e3d0ff9fb0...
39999,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...


In [25]:
result = pd.merge(label, cnt_payment_week, how='outer').fillna(value=0)
result = pd.merge(result, last_payment_week, how='outer').fillna(value=0)
result = pd.merge(result, payment, how='outer')
result = pd.merge(result, payment_amount, how='outer')
result.tail()

,acc_id,cnt_payment_week,last_payment_week,max_payment_amount,total_payment_amount,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8,pay_amount_std
39995,d048b24f5205a50916339d24fc9f7b1c69352d2ee10f1c...,0.0,0.0,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
39996,ad5b3ef19e64aceb9ffea55310607ba62146218e616a83...,0.0,0.0,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
39997,972114a3bb57c377d162fb66b9198a2eec40de9576af51...,0.0,0.0,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
39998,3d10d680df3d5fa2e370997c18274bad7355e3d0ff9fb0...,4.0,8.0,2.662602,3.445700,0.553227,-0.149898,-0.149898,-0.149898,0.553227,-0.149898,0.276238,2.662602,0.954961
39999,813ad8039e2692aecc1b7875f3a9b1e951477937ec30bd...,5.0,8.0,14.510262,25.414101,1.607915,1.748540,-0.149898,-0.149898,14.510262,2.627446,-0.149898,5.369634,4.946280


In [26]:
print(result.shape)

(40000, 14)


In [27]:
result.describe(include="all")

,acc_id,cnt_payment_week,last_payment_week,max_payment_amount,total_payment_amount,pay_amount_w1,pay_amount_w2,pay_amount_w3,pay_amount_w4,pay_amount_w5,pay_amount_w6,pay_amount_w7,pay_amount_w8,pay_amount_std
count,40000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000
unique,40000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,1f47d2f6a59330857f30c156c34bc8cb6c1e2b73a38207...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.414400,0.827550,0.440030,0.002448,-0.059866,-0.077855,0.060532,0.041977,-0.031815,0.029202,0.006298,0.033976,0.216994
std,NaN,1.292755,2.282551,2.257621,4.729582,0.628373,0.502628,1.217024,1.240661,0.833809,1.186921,0.971084,1.080909,0.825977
min,NaN,0.000000,0.000000,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
25%,NaN,0.000000,0.000000,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
50%,NaN,0.000000,0.000000,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000
75%,NaN,0.000000,0.000000,-0.149898,-1.199188,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,-0.149898,0.000000


In [29]:
result.to_csv("../data/test_payment.csv", index=False)